In [1]:
!pip install customtkinter fpdf2 matplotlib


In [3]:
# HMS Major Project - Jupyter Compatible
# Features:
# - Admin + Doctor login (role-based)
# - Patient registration, search, filters
# - Appointment booking
# - Medical records viewer
# - Billing + PDF slips
# - Rooms management
# - Medicine inventory (add/update/use)
# - Dashboard graphs (matplotlib)
# - CSV storage
# - Uses customtkinter, fpdf2, matplotlib
# NOTE: Run in a single Jupyter cell. GUI runs in a separate thread.

import os
import csv
import threading
import datetime
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import customtkinter as ctk

# fpdf and plotting
try:
    from fpdf import FPDF, XPos, YPos
except Exception as e:
    tk.Tk().withdraw()
    messagebox.showerror("Missing package", "Install required packages:\n\npip install fpdf2 customtkinter matplotlib")
    raise

import matplotlib.pyplot as plt

# -------------------------
# File paths
# -------------------------
DATA_DIR = "."
PATIENT_FILE = os.path.join(DATA_DIR, "hms_patients.csv")
APPOINT_FILE = os.path.join(DATA_DIR, "hms_appointments.csv")
BILL_FILE = os.path.join(DATA_DIR, "hms_bills.csv")
USERS_FILE = os.path.join(DATA_DIR, "hms_users.csv")
ROOMS_FILE = os.path.join(DATA_DIR, "hms_rooms.csv")
MEDS_FILE = os.path.join(DATA_DIR, "hms_medicines.csv")

# -------------------------
# Ensure CSVs exist (with headers)
# -------------------------
def ensure_csv(file, header):
    if not os.path.exists(file):
        with open(file, "w", newline="") as f:
            csv.writer(f).writerow(header)

ensure_csv(PATIENT_FILE, ["PatientID","Name","Age","Gender","Disease","Doctor","RegisteredOn"])
ensure_csv(APPOINT_FILE, ["PatientID","Name","Doctor","Date","Time","BookedOn"])
ensure_csv(BILL_FILE, ["PatientID","Name","RoomCharge","DoctorFee","MedicineFee","Total","BilledOn","BillFile"])
ensure_csv(ROOMS_FILE, ["RoomNo","Type","OccupiedBy"])  # OccupiedBy = PatientID or blank
ensure_csv(MEDS_FILE, ["MedID","Name","Qty","Price"])

# Default users (admin & a sample doctor) if not present
if not os.path.exists(USERS_FILE):
    with open(USERS_FILE, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["Username","Password","Role","DisplayName"])
        w.writerow(["admin","admin123","admin","Administrator"])
        w.writerow(["drraj","doctor123","doctor","Dr. Raj"])

# Create some sample rooms & medicines if empty
with open(ROOMS_FILE, "r", newline="") as f:
    rows = list(csv.reader(f))
if len(rows) <= 1:
    with open(ROOMS_FILE, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["RoomNo","Type","OccupiedBy"])
        for i in range(101, 111):
            w.writerow([str(i), "General", ""])

with open(MEDS_FILE, "r", newline="") as f:
    rows = list(csv.reader(f))
if len(rows) <= 1:
    with open(MEDS_FILE, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["MedID","Name","Qty","Price"])
        w.writerow(["M001","Paracetamol","200","2"])
        w.writerow(["M002","Amoxicillin","100","5"])

# -------------------------
# Utility functions
# -------------------------
def read_csv(file):
    with open(file, "r", newline="") as f:
        reader = csv.reader(f)
        return list(reader)

def append_csv(file, row):
    with open(file, "a", newline="") as f:
        csv.writer(f).writerow(row)

def overwrite_csv(file, rows):
    with open(file, "w", newline="") as f:
        csv.writer(f).writerows(rows)

def generate_pdf(title, data_lines, filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Helvetica", "B", 16)
    pdf.cell(0, 10, title, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
    pdf.ln(5)
    pdf.set_font("Helvetica", size=12)
    for line in data_lines:
        pdf.cell(0, 8, line, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
    pdf.output(filename)

# -------------------------
# HMS GUI (runs in thread)
# -------------------------
def start_hms_gui():

    ctk.set_appearance_mode("System")
    ctk.set_default_color_theme("blue")
    app = ctk.CTk()
    app.title("Hospital Management System - Major Project")
    app.geometry("1100x700")
    app.resizable(True, True)

    # global state
    current_user = {"username": None, "role": None, "display": None}

    # -------------------------
    # Left menu frame
    # -------------------------
    sidebar = ctk.CTkFrame(app, width=260)
    sidebar.pack(side="left", fill="y")

    content = ctk.CTkFrame(app)
    content.pack(side="right", expand=True, fill="both")

    # Helper to clear content
    def clear_content():
        for w in content.winfo_children():
            w.destroy()

    # -------------------------
    # LOGIN SCREEN (role-based)
    # -------------------------
    def login_screen():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(padx=20, pady=20, fill="both", expand=True)

        ctk.CTkLabel(frame, text="Login to HMS", font=ctk.CTkFont(size=20, weight="bold")).pack(pady=12)

        username = ctk.CTkEntry(frame, placeholder_text="Username")
        username.pack(pady=8, padx=10)
        password = ctk.CTkEntry(frame, placeholder_text="Password", show="*")
        password.pack(pady=8, padx=10)

        def do_login():
            u = username.get().strip()
            p = password.get().strip()
            users = read_csv(USERS_FILE)
            found = False
            for row in users[1:]:
                if row[0] == u and row[1] == p:
                    found = True
                    current_user["username"] = row[0]
                    current_user["role"] = row[2]
                    current_user["display"] = row[3]
                    break
            if not found:
                messagebox.showerror("Login Failed", "Invalid credentials")
                return
            messagebox.showinfo("Welcome", f"Welcome {current_user['display']} ({current_user['role']})")
            refresh_side_buttons()
            dashboard()

        ctk.CTkButton(frame, text="Login", command=do_login).pack(pady=12)
        # Register new user - only admin can create later from admin panel; but allow creating non-admin for demo:
        def quick_create_doctor():
            nm = simpledialog.askstring("Create Doctor", "Enter username for doctor:")
            if not nm:
                return
            pwd = simpledialog.askstring("Password", "Enter password:")
            if not pwd:
                return
            disp = simpledialog.askstring("Display Name", "Enter display name for doctor:", initialvalue="Doctor")
            append_csv(USERS_FILE, [nm, pwd, "doctor", disp])
            messagebox.showinfo("Created", f"Doctor user {nm} created.")
        ctk.CTkButton(frame, text="Quick Create Doctor (demo)", command=quick_create_doctor).pack(pady=6)

    # -------------------------
    # DASHBOARD (graphs) - available to admin
    # -------------------------
    def dashboard():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Dashboard", font=ctk.CTkFont(size=20, weight="bold")).pack(pady=8)

        # Small stats
        patients = read_csv(PATIENT_FILE)
        appts = read_csv(APPOINT_FILE)
        bills = read_csv(BILL_FILE)
        meds = read_csv(MEDS_FILE)
        rooms = read_csv(ROOMS_FILE)

        stat_frame = ctk.CTkFrame(frame)
        stat_frame.pack(fill="x", padx=10, pady=8)
        ctk.CTkLabel(stat_frame, text=f"Patients: {len(patients)-1}").pack(side="left", padx=12)
        ctk.CTkLabel(stat_frame, text=f"Appointments: {len(appts)-1}").pack(side="left", padx=12)
        ctk.CTkLabel(stat_frame, text=f"Bills: {len(bills)-1}").pack(side="left", padx=12)
        ctk.CTkLabel(stat_frame, text=f"Medicines: {len(meds)-1}").pack(side="left", padx=12)
        occupied = sum(1 for r in rooms[1:] if r[2].strip() != "")
        ctk.CTkLabel(stat_frame, text=f"Occupied Rooms: {occupied}/{len(rooms)-1}").pack(side="left", padx=12)

        # Plot: Patients per day (based on RegisteredOn)
        dates = {}
        for row in patients[1:]:
            d = row[6] if len(row) > 6 else ""
            if not d:
                continue
            dates[d] = dates.get(d, 0) + 1
        if dates:
            x = sorted(dates.keys(), key=lambda s: datetime.datetime.strptime(s, "%Y-%m-%d"))
            y = [dates[k] for k in x]
            plt.figure(figsize=(6,3))
            plt.plot(x, y)
            plt.title("Patients registered per day")
            plt.xlabel("Date")
            plt.ylabel("Count")
            plt.tight_layout()
            plt.show()
        else:
            ctk.CTkLabel(frame, text="No patient registration data to plot yet.").pack(pady=10)

    # -------------------------
    # PATIENT REGISTRATION + SEARCH + FILTERS
    # -------------------------
    def patient_registration_view():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Patient Registration & Search", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=6)

        # Left form
        left = ctk.CTkFrame(frame)
        left.pack(side="left", fill="y", padx=8, pady=8)

        entry_id = ctk.CTkEntry(left, placeholder_text="Patient ID")
        entry_id.pack(pady=6)
        entry_name = ctk.CTkEntry(left, placeholder_text="Name")
        entry_name.pack(pady=6)
        entry_age = ctk.CTkEntry(left, placeholder_text="Age")
        entry_age.pack(pady=6)
        entry_gender = ctk.CTkComboBox(left, values=["Male","Female","Other"])
        entry_gender.set("Male")
        entry_gender.pack(pady=6)
        entry_dis = ctk.CTkEntry(left, placeholder_text="Disease")
        entry_dis.pack(pady=6)
        entry_doc = ctk.CTkEntry(left, placeholder_text="Assigned Doctor")
        entry_doc.pack(pady=6)

        def register_patient():
            pid = entry_id.get().strip()
            name = entry_name.get().strip()
            if not pid or not name:
                messagebox.showerror("Error", "Patient ID and Name required")
                return
            age = entry_age.get().strip()
            gender = entry_gender.get().strip()
            dis = entry_dis.get().strip()
            doctor = entry_doc.get().strip()
            regdate = datetime.date.today().strftime("%Y-%m-%d")
            append_csv(PATIENT_FILE, [pid, name, age, gender, dis, doctor, regdate])
            messagebox.showinfo("Registered", f"Patient {name} registered.")
            refresh_patient_table()

        ctk.CTkButton(left, text="Register Patient", command=register_patient).pack(pady=6)
        ctk.CTkButton(left, text="Clear Fields", command=lambda: [entry_id.delete(0,"end"), entry_name.delete(0,"end"),
                                                                   entry_age.delete(0,"end"), entry_dis.delete(0,"end"),
                                                                   entry_doc.delete(0,"end")]).pack(pady=6)

        # Right table with search and filters
        right = ctk.CTkFrame(frame)
        right.pack(side="right", fill="both", expand=True, padx=6, pady=6)
        search_frame = ctk.CTkFrame(right)
        search_frame.pack(fill="x", pady=6)
        search_entry = ctk.CTkEntry(search_frame, placeholder_text="Search by Name or ID")
        search_entry.pack(side="left", padx=6, pady=6, fill="x", expand=True)
        gender_filter = ctk.CTkComboBox(search_frame, values=["All","Male","Female","Other"])
        gender_filter.set("All")
        gender_filter.pack(side="left", padx=6)
        doctor_filter = ctk.CTkEntry(search_frame, placeholder_text="Filter by Doctor")
        doctor_filter.pack(side="left", padx=6)

        columns = ("PatientID","Name","Age","Gender","Disease","Doctor","RegDate")
        tree = ttk.Treeview(right, columns=columns, show="headings")
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=110)
        tree.pack(fill="both", expand=True)

        def refresh_patient_table():
            for i in tree.get_children():
                tree.delete(i)
            rows = read_csv(PATIENT_FILE)[1:]
            q = search_entry.get().strip().lower()
            gf = gender_filter.get().strip()
            df = doctor_filter.get().strip().lower()
            for r in rows:
                pid, nm, age, gender, dis, doc, regd = r
                if q and (q not in pid.lower() and q not in nm.lower()):
                    continue
                if gf != "All" and gender != gf:
                    continue
                if df and df not in doc.lower():
                    continue
                tree.insert("", "end", values=r)

        ctk.CTkButton(search_frame, text="Search/Filter", command=refresh_patient_table).pack(side="left", padx=6)
        ctk.CTkButton(search_frame, text="Refresh", command=refresh_patient_table).pack(side="left", padx=6)

        # Allow selecting a patient and quick actions
        def on_select_patient():
            sel = tree.focus()
            if not sel:
                messagebox.showerror("Select", "Select a patient row first")
                return
            vals = tree.item(sel)["values"]
            # Show patient details and options
            pid, nm, age, gender, dis, doc, regd = vals
            res = messagebox.askquestion("Patient Action", f"Patient: {nm}\n\nChoose action:\nYes = Book Appointment\nNo = Generate Report PDF")
            if res == "yes":
                # open appointment popup
                appt_date = simpledialog.askstring("Appointment Date", "Enter Date (DD/MM/YYYY):")
                appt_time = simpledialog.askstring("Appointment Time", "Enter Time (e.g., 10:00 AM):")
                if appt_date and appt_time:
                    append_csv(APPOINT_FILE, [pid, nm, doc, appt_date, appt_time, datetime.datetime.now().isoformat()])
                    messagebox.showinfo("Booked", "Appointment booked.")
            else:
                # generate patient summary pdf
                lines = [
                    f"PatientID: {pid}",
                    f"Name: {nm}",
                    f"Age: {age}",
                    f"Gender: {gender}",
                    f"Disease: {dis}",
                    f"Doctor: {doc}",
                    f"Registered On: {regd}"
                ]
                fname = f"Patient_{pid}.pdf"
                generate_pdf("Patient Summary", lines, fname)

        ctk.CTkButton(right, text="Action on Selected", command=on_select_patient).pack(pady=6)
        refresh_patient_table()

    # -------------------------
    # APPOINTMENTS VIEW
    # -------------------------
    def appointment_view():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Appointments", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=8)

        # Table
        cols = ("PatientID","Name","Doctor","Date","Time","BookedOn")
        tree = ttk.Treeview(frame, columns=cols, show="headings")
        for c in cols:
            tree.heading(c, text=c)
            tree.column(c, width=140)
        tree.pack(fill="both", expand=True)

        def refresh():
            for i in tree.get_children():
                tree.delete(i)
            rows = read_csv(APPOINT_FILE)[1:]
            for r in rows:
                tree.insert("", "end", values=r)
        refresh()

        def cancel_appointment():
            sel = tree.focus()
            if not sel:
                messagebox.showerror("Select", "Select appointment")
                return
            vals = tree.item(sel)["values"]
            rows = read_csv(APPOINT_FILE)
            rows = [r for r in rows if r != list(vals)]
            overwrite_csv(APPOINT_FILE, rows)
            refresh()
            messagebox.showinfo("Cancelled", "Appointment cancelled")

        ctk.CTkButton(frame, text="Cancel Selected", fg_color="red", command=cancel_appointment).pack(pady=6)

    # -------------------------
    # BILLING SYSTEM (with room & med usage)
    # -------------------------
    def billing_view():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Billing", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=8)

        entry_pid = ctk.CTkEntry(frame, placeholder_text="Patient ID")
        entry_pid.pack(pady=6)
        entry_name = ctk.CTkEntry(frame, placeholder_text="Patient Name")
        entry_name.pack(pady=6)
        entry_room = ctk.CTkEntry(frame, placeholder_text="Room Charge (number)")
        entry_room.pack(pady=6)
        entry_docfee = ctk.CTkEntry(frame, placeholder_text="Doctor Fee (number)")
        entry_docfee.pack(pady=6)

        # Medicine usage selection - list of medicines with qty
        meds = read_csv(MEDS_FILE)[1:]
        med_map = {m[0]:m for m in meds}  # MedID -> row

        med_frame = ctk.CTkFrame(frame)
        med_frame.pack(pady=6, fill="x")
        ctk.CTkLabel(med_frame, text="Select Medicine and Qty (one by one)").pack(pady=4)
        med_id_cb = ctk.CTkComboBox(med_frame, values=[m[0]+" - "+m[1] for m in meds]) if meds else ctk.CTkComboBox(med_frame, values=[])
        med_id_cb.pack(side="left", padx=6)
        med_qty = ctk.CTkEntry(med_frame, placeholder_text="Qty")
        med_qty.pack(side="left", padx=6)
        selected_meds = []  # list of (MedID, Qty)

        def add_med_to_list():
            val = med_id_cb.get().strip()
            if not val:
                messagebox.showerror("Select", "Select medicine")
                return
            med_id = val.split(" - ")[0]
            try:
                q = int(med_qty.get().strip())
            except:
                messagebox.showerror("Invalid", "Enter integer qty")
                return
            selected_meds.append((med_id, q))
            messagebox.showinfo("Added", f"{med_id} qty {q} added")
            med_qty.delete(0,"end")

        ctk.CTkButton(med_frame, text="Add", command=add_med_to_list).pack(side="left", padx=6)

        def create_bill_and_pdf():
            pid = entry_pid.get().strip()
            name = entry_name.get().strip()
            try:
                roomc = int(entry_room.get().strip())
                docf = int(entry_docfee.get().strip())
            except:
                messagebox.showerror("Invalid", "Room and doctor fee must be numbers")
                return
            # Calculate med fees and update stock
            med_total = 0
            meds_rows = read_csv(MEDS_FILE)
            for mid, qty in selected_meds:
                # find med row
                for i, row in enumerate(meds_rows):
                    if i==0: continue
                    if row[0] == mid:
                        stock = int(row[2])
                        price = float(row[3])
                        if qty > stock:
                            messagebox.showerror("Stock", f"Not enough stock for {mid}")
                            return
                        # deduct
                        meds_rows[i][2] = str(stock - qty)
                        med_total += qty * price
                        break
            # write meds file
            overwrite_csv(MEDS_FILE, meds_rows)
            total = roomc + docf + med_total
            bill_fname = f"Bill_{pid}_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.pdf"
            append_csv(BILL_FILE, [pid, name, str(roomc), str(docf), str(med_total), str(total), datetime.datetime.now().strftime("%Y-%m-%d"), bill_fname])
            lines = [
                f"PatientID: {pid}",
                f"Name: {name}",
                f"Room Charge: {roomc}",
                f"Doctor Fee: {docf}",
                f"Medicine Fee: {round(med_total,2)}",
                f"Total: {round(total,2)}"
            ]
            generate_pdf("Hospital Bill", lines, bill_fname)
            messagebox.showinfo("Billed", f"Bill generated: {bill_fname}")

        ctk.CTkButton(frame, text="Create Bill & Generate PDF", command=create_bill_and_pdf).pack(pady=6)

    # -------------------------
    # ROOMS MANAGEMENT
    # -------------------------
    def rooms_view():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Rooms Management", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=8)

        tree = ttk.Treeview(frame, columns=("RoomNo","Type","OccupiedBy"), show="headings")
        for c in ("RoomNo","Type","OccupiedBy"):
            tree.heading(c, text=c)
            tree.column(c, width=150)
        tree.pack(fill="both", expand=True)

        def refresh():
            for i in tree.get_children():
                tree.delete(i)
            rows = read_csv(ROOMS_FILE)[1:]
            for r in rows:
                tree.insert("", "end", values=r)
        refresh()

        def assign_room():
            sel = tree.focus()
            if not sel:
                messagebox.showerror("Select", "Select room row")
                return
            room = tree.item(sel)["values"][0]
            pid = simpledialog.askstring("Assign", "Enter PatientID to assign to this room:")
            if not pid:
                return
            rows = read_csv(ROOMS_FILE)
            for i,r in enumerate(rows):
                if i==0: continue
                if r[0] == room:
                    rows[i][2] = pid
            overwrite_csv(ROOMS_FILE, rows)
            refresh()
            messagebox.showinfo("Assigned", f"Room {room} assigned to {pid}")

        def free_room():
            sel = tree.focus()
            if not sel:
                messagebox.showerror("Select", "Select room row")
                return
            room = tree.item(sel)["values"][0]
            rows = read_csv(ROOMS_FILE)
            for i,r in enumerate(rows):
                if i==0: continue
                if r[0] == room:
                    rows[i][2] = ""
            overwrite_csv(ROOMS_FILE, rows)
            refresh()
            messagebox.showinfo("Freed", f"Room {room} freed")

        ctk.CTkButton(frame, text="Assign Selected Room", command=assign_room).pack(pady=6)
        ctk.CTkButton(frame, text="Free Selected Room", fg_color="red", command=free_room).pack(pady=6)

    # -------------------------
    # MEDICINE INVENTORY
    # -------------------------
    def meds_view():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Medicine Inventory", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=8)

        top = ctk.CTkFrame(frame)
        top.pack(fill="x", padx=6, pady=6)
        med_id = ctk.CTkEntry(top, placeholder_text="MedID (e.g., M010)")
        med_id.pack(side="left", padx=6)
        med_name = ctk.CTkEntry(top, placeholder_text="Name")
        med_name.pack(side="left", padx=6)
        med_qty = ctk.CTkEntry(top, placeholder_text="Qty")
        med_qty.pack(side="left", padx=6)
        med_price = ctk.CTkEntry(top, placeholder_text="Price")
        med_price.pack(side="left", padx=6)

        def add_med():
            mid = med_id.get().strip()
            if not mid:
                messagebox.showerror("Error", "MedID required")
                return
            try:
                q = int(med_qty.get().strip())
                p = float(med_price.get().strip())
            except:
                messagebox.showerror("Error", "Qty numeric & Price numeric")
                return
            rows = read_csv(MEDS_FILE)
            # check exists
            exists = False
            for i,r in enumerate(rows):
                if i==0: continue
                if r[0] == mid:
                    rows[i][2] = str(int(rows[i][2]) + q)
                    rows[i][3] = str(p)
                    exists = True
                    break
            if not exists:
                rows.append([mid, med_name.get().strip(), str(q), str(p)])
            overwrite_csv(MEDS_FILE, rows)
            messagebox.showinfo("Added", f"Medicine {mid} added/updated")
            refresh_meds()

        top_btn = ctk.CTkButton(top, text="Add/Update", command=add_med)
        top_btn.pack(side="left", padx=6)

        tree = ttk.Treeview(frame, columns=("MedID","Name","Qty","Price"), show="headings")
        for c in ("MedID","Name","Qty","Price"):
            tree.heading(c, text=c)
            tree.column(c, width=120)
        tree.pack(fill="both", expand=True, padx=6, pady=6)

        def refresh_meds():
            for i in tree.get_children():
                tree.delete(i)
            rows = read_csv(MEDS_FILE)[1:]
            for r in rows:
                tree.insert("", "end", values=r)
        refresh_meds()

        def delete_med():
            sel = tree.focus()
            if not sel:
                messagebox.showerror("Select", "Select med")
                return
            vals = tree.item(sel)["values"]
            mids = vals[0]
            rows = read_csv(MEDS_FILE)
            rows = [r for r in rows if r[0] != mids]
            overwrite_csv(MEDS_FILE, rows)
            refresh_meds()
            messagebox.showinfo("Deleted", "Medicine removed")

        ctk.CTkButton(frame, text="Delete Selected", fg_color="red", command=delete_med).pack(pady=6)

    # -------------------------
    # USERS/Admin panel (only admin)
    # -------------------------
    def users_view():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Users (Admin Only)", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=8)

        tree = ttk.Treeview(frame, columns=("Username","Role","Display"), show="headings")
        for c in ("Username","Role","Display"):
            tree.heading(c, text=c)
            tree.column(c, width=200)
        tree.pack(fill="both", expand=True)

        def refresh():
            for i in tree.get_children():
                tree.delete(i)
            rows = read_csv(USERS_FILE)[1:]
            for r in rows:
                tree.insert("", "end", values=[r[0], r[2], r[3]])
        refresh()

        def add_user():
            u = simpledialog.askstring("Username","Enter username:")
            p = simpledialog.askstring("Password","Enter password:")
            role = simpledialog.askstring("Role","admin or doctor:")
            display = simpledialog.askstring("Display name","Display name:")
            if not u or not p or not role:
                return
            append_csv(USERS_FILE, [u,p,role,display])
            refresh()
            messagebox.showinfo("Added", "User added")

        def delete_user():
            sel = tree.focus()
            if not sel:
                messagebox.showerror("Select", "Select user")
                return
            vals = tree.item(sel)["values"]
            uname = vals[0]
            rows = read_csv(USERS_FILE)
            rows = [r for r in rows if r[0] != uname]
            overwrite_csv(USERS_FILE, rows)
            refresh()
            messagebox.showinfo("Deleted", "User removed")

        ctk.CTkButton(frame, text="Add User", command=add_user).pack(pady=6)
        ctk.CTkButton(frame, text="Delete Selected", fg_color="red", command=delete_user).pack(pady=6)

    # -------------------------
    # BILL HISTORY VIEW
    # -------------------------
    def bills_view():
        clear_content()
        frame = ctk.CTkFrame(content)
        frame.pack(fill="both", expand=True, padx=12, pady=12)
        ctk.CTkLabel(frame, text="Bills", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=8)

        tree = ttk.Treeview(frame, columns=("PatientID","Name","Room","DoctorFee","MedFee","Total","Date","File"), show="headings")
        for c in ("PatientID","Name","Room","DoctorFee","MedFee","Total","Date","File"):
            tree.heading(c, text=c)
            tree.column(c, width=120)
        tree.pack(fill="both", expand=True)

        def refresh():
            for i in tree.get_children():
                tree.delete(i)
            rows = read_csv(BILL_FILE)[1:]
            for r in rows:
                tree.insert("", "end", values=r)
        refresh()

        def open_pdf():
            sel = tree.focus()
            if not sel:
                messagebox.showerror("Select", "Select bill")
                return
            vals = tree.item(sel)["values"]
            fname = vals[-1]
            if os.path.exists(fname):
                os.startfile(fname) if os.name == 'nt' else os.system(f'xdg-open "{fname}"')
            else:
                messagebox.showerror("Missing", "PDF file not found")

        ctk.CTkButton(frame, text="Open Selected PDF", command=open_pdf).pack(pady=6)

    # -------------------------
    # Side buttons (update based on role)
    # -------------------------
    buttons = {}

    def add_side_button(name, cmd):
        b = ctk.CTkButton(sidebar, text=name, width=220, command=cmd)
        b.pack(pady=10)
        buttons[name] = b

    # Create buttons for all features but they will check role inside if needed
    add_side_button("Login", login_screen)
    add_side_button("Dashboard", dashboard)
    add_side_button("Patients", patient_registration_view)
    add_side_button("Appointments", appointment_view)
    add_side_button("Billing", billing_view)
    add_side_button("Rooms", rooms_view)
    add_side_button("Medicines", meds_view)
    add_side_button("Bills History", bills_view)
    add_side_button("Users (Admin)", users_view)
    add_side_button("Logout", lambda: do_logout())

    # Role control
    def refresh_side_buttons():
        role = current_user.get("role")
        # Default disable all, then enable appropriate
        for name, btn in buttons.items():
            btn.configure(state="disabled")
        buttons["Login"].configure(state="normal")
        if role == "admin":
            # admin can access all
            for name in buttons:
                buttons[name].configure(state="normal")
        elif role == "doctor":
            # doctor limited
            for name in ["Dashboard","Patients","Appointments","Billing","Bills History","Rooms","Medicines","Logout"]:
                if name in buttons:
                    buttons[name].configure(state="normal")
            # doctor should not manage users
            if "Users (Admin)" in buttons:
                buttons["Users (Admin)"].configure(state="disabled")
            # login disabled
            buttons["Login"].configure(state="disabled")
        else:
            # not logged in
            buttons["Login"].configure(state="normal")
            buttons["Logout"].configure(state="disabled")

    def do_logout():
        current_user["username"] = None
        current_user["role"] = None
        current_user["display"] = None
        messagebox.showinfo("Logged out", "You have been logged out")
        refresh_side_buttons()
        login_screen()

    # initial
    refresh_side_buttons()
    login_screen()

    app.mainloop()

# Run GUI in a thread to keep Jupyter responsive
threading.Thread(target=start_hms_gui, daemon=True).start()
